In [11]:
%matplotlib inline
# %matplotlib notebook
!git pull
# import os;os.environ['LAMBDA_PARQUET_TICK_DB']=rf'X:\lambda_data'
import os;os.environ['LAMBDA_PARQUET_TICK_DB']=rf'X:'

Already up to date.


In [12]:
from notebooks.lambda_imports import *
import time
import datetime
import tqdm
import seaborn as sns
sns.set_theme()

# from hurst import compute_Hc, random_walk
# plot_with_dark_style()

In [13]:
FIRST_HOUR=7.0
LAST_HOUR=19.0
tick = TickDB()
instrument_pk='btcusdt_binance'

start_dates=[]
end_dates=[]

start_date_fi=datetime.datetime(year=2021, day=22, month=12,hour=int(FIRST_HOUR))
end_date_fi=datetime.datetime(year=2021, day=22, month=12,hour=int(LAST_HOUR))

start_date_train=datetime.datetime(year=2021, day=23, month=12,hour=int(FIRST_HOUR))
end_date_train=datetime.datetime(year=2021, day=23, month=12,hour=int(LAST_HOUR))

start_dates=[start_date_fi,start_date_train]
end_dates=[end_date_fi,end_date_train]

start_date_tests=[]
end_date_tests=[]

days_test=[]
testing_days=31
def get_time(reference_date,business_days,hour):
    from pandas.tseries.offsets import BDay
#     date=(reference_date-BDay(business_days))
    date=reference_date-datetime.timedelta(days = business_days)
    return datetime.datetime.fromtimestamp(date.timestamp()).replace(hour=hour, minute=0,second=0,microsecond=0)

for day in range(1,testing_days):
    day_to_add=get_time(start_date_train,-day,0)
    if day_to_add.day==25:#we dont have data
        continue
    days_test.append(day_to_add)
days_test

for day in days_test:
    start_date = day+ datetime.timedelta(hours=int(FIRST_HOUR))
    end_date=day+ datetime.timedelta(hours=int(LAST_HOUR))
    start_date_tests.append(start_date)
    end_date_tests.append(end_date)
    
start_dates+=start_date_tests
end_dates+=end_date_tests

start_dates

[datetime.datetime(2021, 12, 22, 7, 0),
 datetime.datetime(2021, 12, 23, 7, 0),
 datetime.datetime(2021, 12, 24, 7, 0),
 datetime.datetime(2021, 12, 26, 7, 0),
 datetime.datetime(2021, 12, 27, 7, 0),
 datetime.datetime(2021, 12, 28, 7, 0),
 datetime.datetime(2021, 12, 29, 7, 0),
 datetime.datetime(2021, 12, 30, 7, 0),
 datetime.datetime(2021, 12, 31, 7, 0),
 datetime.datetime(2022, 1, 1, 7, 0),
 datetime.datetime(2022, 1, 2, 7, 0),
 datetime.datetime(2022, 1, 3, 7, 0),
 datetime.datetime(2022, 1, 4, 7, 0),
 datetime.datetime(2022, 1, 5, 7, 0),
 datetime.datetime(2022, 1, 6, 7, 0),
 datetime.datetime(2022, 1, 7, 7, 0),
 datetime.datetime(2022, 1, 8, 7, 0),
 datetime.datetime(2022, 1, 9, 7, 0),
 datetime.datetime(2022, 1, 10, 7, 0),
 datetime.datetime(2022, 1, 11, 7, 0),
 datetime.datetime(2022, 1, 12, 7, 0),
 datetime.datetime(2022, 1, 13, 7, 0),
 datetime.datetime(2022, 1, 14, 7, 0),
 datetime.datetime(2022, 1, 15, 7, 0),
 datetime.datetime(2022, 1, 16, 7, 0),
 datetime.datetime(2022, 

In [14]:
#pt
depth_df=None
trade_df=None
bins=['5S','1T','1H']#5 secondss , 1 minute
df={}
ts_df={}
for i in range(len(start_dates)):
# for i in range(3):
    depth_df_it = tick.get_depth(instrument_pk=instrument_pk,start_date=start_dates[i],end_date=end_dates[i]).drop_duplicates()[start_dates[i]:end_dates[i]]
    trade_df_it = tick.get_trades(instrument_pk=instrument_pk,start_date=start_dates[i],end_date=end_dates[i]).drop_duplicates()[start_dates[i]:end_dates[i]]
    date = start_dates[i].replace(hour=0)
    
    df[date]=[depth_df_it,trade_df_it,len(depth_df_it),len(trade_df_it),len(depth_df_it)+len(trade_df_it)]
    for bin_it in bins:
        
        #ts tick counts
        ts = depth_df_it[depth_df_it.columns[0]].resample(bin_it).count()+trade_df_it[trade_df_it.columns[0]].resample(bin_it).count()
        ts_df_bin = pd.DataFrame(ts,columns=[date])
        
        #remove days from index
        ts_df_bin = ts_df_bin.reset_index()
        ts_df_bin['date'] = ts_df_bin['date'].apply(lambda x: x.strftime('%X'))
        ts_df_bin.set_index('date', inplace=True)
        
        if bin_it not in ts_df.keys():
            ts_df[bin_it]=ts_df_bin
        else:
             ts_df[bin_it][date]= ts_df_bin
        
        #summaries
        df[date]+=[
            depth_df_it[depth_df_it.columns[0]].resample(bin_it).count().mean(),
            trade_df_it[trade_df_it.columns[0]].resample(bin_it).count().mean(),
            depth_df_it[depth_df_it.columns[0]].resample(bin_it).count().mean()+ trade_df_it[trade_df_it.columns[0]].resample(bin_it).count().mean()
           ]
        
    if depth_df is None:
        depth_df=depth_df_it
    else:
        depth_df=depth_df.append(depth_df_it)
        
    if trade_df is None:
        trade_df=trade_df_it
    else:
        trade_df=trade_df.append(trade_df_it) 

downloading btcusdt_binance depth from 20211222 to 20211222
downloading btcusdt_binance trade from 20211222 to 20211222
downloading btcusdt_binance depth from 20211223 to 20211223
downloading btcusdt_binance trade from 20211223 to 20211223
downloading btcusdt_binance depth from 20211224 to 20211224
downloading btcusdt_binance trade from 20211224 to 20211224
downloading btcusdt_binance depth from 20211226 to 20211226
downloading btcusdt_binance trade from 20211226 to 20211226
downloading btcusdt_binance depth from 20211227 to 20211227
downloading btcusdt_binance trade from 20211227 to 20211227
downloading btcusdt_binance depth from 20211228 to 20211228
downloading btcusdt_binance trade from 20211228 to 20211228
downloading btcusdt_binance depth from 20211229 to 20211229
downloading btcusdt_binance trade from 20211229 to 20211229
downloading btcusdt_binance depth from 20211230 to 20211230
downloading btcusdt_binance trade from 20211230 to 20211230
downloading btcusdt_binance depth from 2

In [15]:
df = pd.DataFrame.from_dict(df).T
# df.to_csv('tick_db_backup.csv')

In [16]:
ts_df[bin_it]

,2021-12-22,2021-12-23,2021-12-24,2021-12-26,2021-12-27,2021-12-28,2021-12-29,2021-12-30,2021-12-31,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,2022-01-11,2022-01-12,2022-01-13,2022-01-14,2022-01-15,2022-01-16,2022-01-17,2022-01-18,2022-01-19,2022-01-20,2022-01-21,2022-01-22
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
07:00:00,22614.0,28201.0,NaN,NaN,21631.0,21548.0,23866.0,21754.0,26357.0,18736.0,17359.0,18608.0,23499.0,19066.0,25704.0,24617.0,19328.0,16814.0,17118.0,19871.0,24386.0,21587.0,22800.0,18328.0,18397.0,20937.0,22211.0,34570.0,18128.0,34368.0,NaN
08:00:00,27895.0,26833.0,10183.0,NaN,24829.0,25732.0,23460.0,24929.0,54663.0,18310.0,23328.0,21561.0,25352.0,21581.0,32291.0,46194.0,22989.0,21217.0,21488.0,32789.0,23459.0,27123.0,28456.0,16928.0,24624.0,22833.0,32659.0,28922.0,18635.0,34374.0,NaN
09:00:00,30572.0,24693.0,5381.0,NaN,25277.0,27840.0,31218.0,21849.0,32299.0,19748.0,22787.0,23897.0,21232.0,27030.0,43319.0,30633.0,20160.0,20772.0,19215.0,23046.0,26136.0,24658.0,20378.0,18437.0,24061.0,23374.0,32360.0,23920.0,19425.0,29382.0,NaN
10:00:00,25831.0,25094.0,NaN,NaN,22739.0,32671.0,26714.0,20712.0,27439.0,23451.0,18261.0,25801.0,24318.0,27295.0,35306.0,27883.0,23307.0,20833.0,21032.0,29689.0,27811.0,24375.0,21468.0,20547.0,22425.0,23836.0,19862.0,23161.0,20016.0,28629.0,4925.0
11:00:00,27848.0,27507.0,NaN,NaN,23277.0,27812.0,25150.0,30018.0,25633.0,19857.0,18340.0,25854.0,27145.0,28564.0,30107.0,24824.0,21222.0,20320.0,27622.0,26589.0,16534.0,28443.0,25006.0,19327.0,25326.0,21901.0,22920.0,32300.0,21673.0,30167.0,45798.0
12:00:00,40598.0,28894.0,NaN,NaN,33615.0,28505.0,39134.0,32149.0,24847.0,19778.0,24524.0,29988.0,24363.0,33376.0,33239.0,24595.0,22810.0,18925.0,27183.0,25165.0,24895.0,23103.0,28409.0,30064.0,21881.0,29946.0,22414.0,26484.0,24426.0,61194.0,47651.0
13:00:00,28793.0,29842.0,NaN,NaN,34429.0,32624.0,32837.0,26668.0,22746.0,18198.0,21285.0,24650.0,26873.0,26938.0,35828.0,32190.0,19936.0,20764.0,3599.0,27265.0,33998.0,23424.0,25855.0,23971.0,12724.0,25714.0,29171.0,28657.0,29125.0,47620.0,41588.0
14:00:00,29655.0,27228.0,NaN,NaN,39669.0,25675.0,34034.0,26866.0,26753.0,18222.0,25343.0,32743.0,18619.0,35466.0,38649.0,33460.0,18538.0,18480.0,3600.0,36608.0,27989.0,28724.0,18351.0,25254.0,3583.0,22298.0,33527.0,33788.0,43258.0,44207.0,37351.0
15:00:00,26929.0,26627.0,NaN,22147.0,28645.0,7814.0,28159.0,23560.0,24250.0,24895.0,26537.0,34336.0,3587.0,29646.0,33435.0,48592.0,27472.0,24750.0,19169.0,36701.0,28507.0,43657.0,3599.0,22632.0,8183.0,23004.0,28478.0,34028.0,48938.0,46580.0,44864.0


Por otro lado, creo que no caracterizamos en ninguna parte el flujo de ticks que se da en los datos de prueba 
(cuántos ticks se producen de media en 5 segundos, cuánto varía esa media durante un día de test, durante los 30 días, …). Creo que convendría hacerlo. (A lo mejor incluso nos da una pista de por qué hay un día en el que los Alpha-AS tienen malos resultados. el 21 diciembre)


In [17]:
# df=pd.read_csv('tick_db_backup.csv',index_col=0)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
2021-12-22,bidQuantity0 bidQuan...,quantity price da...,42737,301320,344057,4.94641,34.871,39.8174,59.3569,417.92,477.277,3561.42,23178.5,26739.9
2021-12-23,bidQuantity0 bidQuan...,quantity price da...,42542,356891,399433,4.92384,41.3068,46.2307,59.0861,495.682,554.768,3545.17,29740.9,33286.1
2021-12-24,bidQuantity0 bidQuan...,quantity price da...,8210,10731,18941,4.80398,13.2809,18.0849,57.4126,157.809,215.221,2736.67,5365.5,8102.17
2021-12-26,bidQuantity0 bidQuan...,quantity price da...,12660,79055,91715,4.50374,28.1134,32.6172,53.8723,334.979,388.851,3165,15811,18976
2021-12-27,bidQuantity0 bidQuan...,quantity price da...,42383,311989,354372,4.90544,36.1098,41.0153,58.8653,433.318,492.183,3531.92,25999.1,29531
2021-12-28,bidQuantity0 bidQuan...,quantity price da...,42820,285603,328423,4.95602,33.0559,38.0119,59.4722,396.121,455.593,3568.33,21969.5,25537.8
2021-12-29,bidQuantity0 bidQuan...,quantity price da...,42372,290749,333121,4.90417,33.6515,38.5557,58.85,403.818,462.668,3531,24229.1,27760.1
2021-12-30,bidQuantity0 bidQuan...,quantity price da...,42190,261745,303935,4.8831,30.2911,35.1742,58.5972,363.031,421.628,3515.83,20134.2,23650.1
2021-12-31,bidQuantity0 bidQuan...,quantity price da...,42699,301030,343729,4.94201,34.8374,39.7794,59.3042,417.517,476.822,3558.25,23156.2,26714.4
2022-01-01,bidQuantity0 bidQuan...,quantity price da...,41323,220944,262267,4.78275,25.5722,30.355,57.3931,306.867,364.26,3443.58,18412,21855.6


In [18]:
summary_df=df.loc[:,2:]
columns_rename = {2:"count_depth",3:"count_trades",4:"count_ticks"}
summary_df=summary_df.rename(columns=columns_rename)
iterator=int(list(columns_rename.keys())[-1])
for bin_it in bins:
#     summary_df=summary_df.rename(columns={iterator:f"df_count_depth_{bin_it}",iterator+1:f"df_count_trade_{bin_it}",iterator+2:f"df_count_ticks_{bin_it}"})
    summary_df=summary_df.rename(columns={iterator+1:f"count_depth_{bin_it}",iterator+2:f"count_trade_{bin_it}",iterator+3:f"count_ticks_{bin_it}"})
    iterator+=3
# summary_df["ticks_average_per_second"]=(summary_df["count_depth"]+summary_df["count_trades"])/((LAST_HOUR-FIRST_HOUR)*60*60)
# summary_df["ticks_average_per_5seconds"]=summary_df["ticks_average_per_second"]*5
# summary_df.fillna(summary_df.mean(),inplace=True)
summary_df
#### 21 dec is the worst dd day

,count_depth,count_trades,count_ticks,count_depth_5S,count_trade_5S,count_ticks_5S,count_depth_1T,count_trade_1T,count_ticks_1T,count_depth_1H,count_trade_1H,count_ticks_1H
2021-12-22,42737,301320,344057,4.94641,34.871,39.8174,59.3569,417.92,477.277,3561.42,23178.5,26739.9
2021-12-23,42542,356891,399433,4.92384,41.3068,46.2307,59.0861,495.682,554.768,3545.17,29740.9,33286.1
2021-12-24,8210,10731,18941,4.80398,13.2809,18.0849,57.4126,157.809,215.221,2736.67,5365.5,8102.17
2021-12-26,12660,79055,91715,4.50374,28.1134,32.6172,53.8723,334.979,388.851,3165,15811,18976
2021-12-27,42383,311989,354372,4.90544,36.1098,41.0153,58.8653,433.318,492.183,3531.92,25999.1,29531
2021-12-28,42820,285603,328423,4.95602,33.0559,38.0119,59.4722,396.121,455.593,3568.33,21969.5,25537.8
2021-12-29,42372,290749,333121,4.90417,33.6515,38.5557,58.85,403.818,462.668,3531,24229.1,27760.1
2021-12-30,42190,261745,303935,4.8831,30.2911,35.1742,58.5972,363.031,421.628,3515.83,20134.2,23650.1
2021-12-31,42699,301030,343729,4.94201,34.8374,39.7794,59.3042,417.517,476.822,3558.25,23156.2,26714.4
2022-01-01,41323,220944,262267,4.78275,25.5722,30.355,57.3931,306.867,364.26,3443.58,18412,21855.6


In [19]:
columns=['count_ticks']
for bin_it in bins:
    columns.append(rf"count_ticks_{bin_it}")
summary_ticks_df=summary_df[columns]
summary_ticks_df

,count_ticks,count_ticks_5S,count_ticks_1T,count_ticks_1H
2021-12-22,344057,39.8174,477.277,26739.9
2021-12-23,399433,46.2307,554.768,33286.1
2021-12-24,18941,18.0849,215.221,8102.17
2021-12-26,91715,32.6172,388.851,18976
2021-12-27,354372,41.0153,492.183,29531
2021-12-28,328423,38.0119,455.593,25537.8
2021-12-29,333121,38.5557,462.668,27760.1
2021-12-30,303935,35.1742,421.628,23650.1
2021-12-31,343729,39.7794,476.822,26714.4
2022-01-01,262267,30.355,364.26,21855.6


In [23]:
summary_df.to_csv("ticks_thesis.csv")

# TimeSeries analysis

In [24]:
for bin_it in bins:
    ts_df[bin_it].to_csv(rf"ticks_thesis_ts_{bin_it}.csv")